In [1]:
#!/usr/bin/env python
# 15-Puzzle-Solver.ipynb : Second Assignment - Solve a variant of 15 Puzzle problem
# 
# Problem Description:
# Instead of sliding a single tile from one cell into an empty cell, in this variant, either one, two, or three tiles  
# may be slid left, right, up or down in a single move
# 
# The goal is to find a short sequence of moves that restores the canonical configuration given an initial board 
# configuration using A* search.
#
# Output Format:
# last line of output should be a representation of the solution path you found, in this format:
# [move-1] [move-2] ... [move-n]
# where each move is encoded as a letter L, R, U, or D for left, right, up, or down, respectively, 
# followed by 1, 2, or 3 indicating the number of tiles to move, 
# followed by a row or column number (indexed beginning at 1).
#
#
# # g_n = cost from start node to node n
# # h_n = estimated cost of cheapest path from a node n to goal

import sys
import time
from copy import deepcopy

In [2]:
class Astar:
    def __init__(self):
        self.initial_board = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
        self.h1 = 0
        self.h2 = 0
        self.init_positions = self.positions(self.initial_board)
        self.city_distances = {}
        self.closed_list = []
        self.goal_parity = 0
        
    # check the parity of the given board - to see if its solvable    
    def check_parity(self, board):
        # get 1D list
        single_list = []
        for row in board:
            for c in row:
                single_list.append(c)
        # get parity sum
        parity = 0
        for i in range(16):
            count = 0
            if single_list[i] == 0:
                continue
            for j in range(i + 1, 16):
                if i ==j or single_list[j] == 0:
                    continue
                if single_list[i] > single_list[j]:
                    count += 1
            parity += count
        return self.goal_parity == parity % 2
    
    # positions on board
    def positions(self, board):
        position = {}
        for i, r in enumerate(board):
            for j, c in enumerate(r):
                position[c] = [i, j]
        return position
        
    # goal function
    def is_goal(self, board):
        return sum([1 for r1, r2 in zip(self.initial_board, board) \
                    for c1, c2 in zip(r1, r2) if c1 == c2]) == 16
    
    # Heuristic function H, manhattan distance
    def heuristic(self, board):
        self.h1 = 15 - sum([1 for r1, r2 in zip(self.initial_board, board) \
                    for c1, c2 in zip(r1, r2) if c1 == c2 and c1 != 0])
        
        curr_positions = self.positions(board)
        for key in range(16):
            [x2, y2], [x1, y1] = curr_positions[key], self.init_positions[key]
            self.city_distances[key] = abs(x2 - x1) + abs(y2 - y1)
        self.h2 = sum(self.city_distances.values())
    
    # 0 location in any board configuration
    def index0(self, board):
        return self.positions(board)[0]
    
    # successor function, 0 can move either up or down, left or right 0 - 3 slides, 
    # accomodates a-star cost function
    def successors_g_hcost(self, given_board):
        board = given_board[0]
        [r, c] = self.index0(board)
        
        # Row successors
        row_successors = []  
        next_board, count, steps = deepcopy(board), 0, given_board[2]
        for i in range(c, 0, -1):
            count += 1
            next_board[r][i] = next_board[r][i - 1]
            next_board[r][i - 1] = 0
            add_board = deepcopy(next_board)
            if add_board in self.closed_list:
                continue
            self.heuristic(add_board)
            row_successors.append([add_board, count + self.h2, steps + " " + str(count) + "L"])
        next_board, count, steps = deepcopy(board), 0, given_board[2]
        for i in range(c, 3):
            count += 1
            next_board[r][i] = next_board[r][i + 1]
            next_board[r][i + 1] = 0
            add_board = deepcopy(next_board)
            if add_board in self.closed_list:
                continue
            self.heuristic(add_board)
            row_successors.append([add_board, count + self.h2, steps + " " + str(count) + "R"])

        # Column successors
        col_successors = []  
        next_board, count, steps = deepcopy(board), 0, given_board[2]
        for i in range(r, 0, -1):
            count += 1
            next_board[i][c] = next_board[i - 1][c]
            next_board[i - 1][c] = 0
            add_board = deepcopy(next_board)
            if add_board in self.closed_list:
                continue
            self.heuristic(add_board)
            col_successors.append([add_board, count + self.h2, steps + " " + str(count) + "U"])
        next_board, count, steps = deepcopy(board), 0, given_board[2]
        for i in range(r, 3):
            count += 1
            next_board[i][c] = next_board[i + 1][c]
            next_board[i + 1][c] = 0
            add_board = deepcopy(next_board)
            if add_board in self.closed_list:
                continue
            self.heuristic(add_board)
            col_successors.append([add_board, count + self.h2, steps + " " + str(count) + "D"])

        successors = sorted(row_successors + col_successors,key=lambda x: x[1], reverse = True)
        return successors
    
    # solver for 15 puzzle
    def solve(self, board):
        if self.check_parity(board):
            self.heuristic(board)
            curr_boards = [[board, self.h2, ""]]
            # count = 0
            while len(curr_boards) > 0:
                for s in self.successors_g_hcost(curr_boards.pop(0)):
                    # count +=1
                    if self.is_goal(s[0]):
                        return s[2]
                    self.closed_list.append(s[0])
                    curr_boards.append(s)
                # curr_boards = sorted(curr_boards,key=lambda x: x[1], reverse = True)
                # print(count)
            print("Sorry no solution found !")
        else:
            print("This board configuration has no solution !")
        
            

In [3]:
astar = Astar()

In [4]:
board = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [0, 13, 14, 15]]
solution = astar.solve(board)
solution

' 3R'

In [5]:
board1 = [[5,1,7,3],[9,2,11,4],[13,6,15,8],[0,10,14,12]]
solution1 = astar.solve(board1)
solution1

' 3U 1R 3D 1R 3U 1R 3D'